## Stepwise guide for running mcmicro on O2

### Step: 1
Upload the raw data to O2 and make sure the data is organized in the following format. Raw data from each slide/image needs to be in its own folder named `raw`. All the images need to be placed under a higher level folder (Dataset). 

<br></br>
```
-- Main_Folder (Dataset)  
    -- Image1/  
        -- raw/  
    -- Image2/  
        -- raw/ 
```

## Step: 2
Create a new file called `submission.sh` and copy paste the following code block into it (This can be anywhere you like). 

<br></br>
New file can be created by using any text editor. I use `vim`.  Just type `vim submission.sh` press `I` to insert or start typing. Paste the following code block and save by pressing `esc` and then `:x` + `enter`

```
#!/bin/sh

# Using getopts for passing in arguments via basg
while getopts ":p:t:" opt; do
  case $opt in
    p) p="$OPTARG"
    ;;
    t) t="$OPTARG"
    ;;
  esac
done

# store sub dirs
SLIDEDIRS=($p/*/)

# loop and submit all jobs
for dir in ${SLIDEDIRS[@]}; do
    cd $dir
    echo $(basename $dir) $dir
    sbatch --job-name $(basename $dir) \
        $t $dir 
    sleep 10
done
```

## Step: 3

Create a template to run mcmicro on all images. This contains all the information of the various settings to be used when running mcmicro.

#### Some fields need to be replaced based on your situation
1. Your email at `#SBATCH --mail-user= ajitj_nirmal@dfci.harvard.edu`
2. `#SBATCH -t 0-12:00` This depends on the size of your dataset. Note if you change this, you might need to change `#SBATCH -p short` as well.
```
Maximum time in each partition
short: 12 hours  
medium: 5 days  
long: 30 days  
```

As before create a new file called `submit_mcmicro.sh` and copy paste the following code block into it.

Just type `vim submit_mcmicro.sh` press `I` to insert or start typing. Paste the following code block and save by pressing `esc` and then `:x` + `enter`

```
#!/bin/bash
#SBATCH -p short
#SBATCH -J nextflow_O2
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 0-12:00
#SBATCH --mem=8G
#SBATCH --mail-type=END
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu

# get the values
SAMPLEDIR=$1
SAMPLEID=$(basename $SAMPLEDIR)

# load packages
module purge
module load java

# Run
nextflow run labsyspharm/mcmicro \
    -profile        O2 \
    --in            $SAMPLEDIR \
    -w              /n/scratch3/users/${USER:0:1}/$USER/work \
    -with-report    $USER-$(date -Is)-$SAMPLEID.html \
    --start-at      illumination \
    --unmicst-opts  '--channel 1' \
    --s3seg-opts    '--cytoMethod ring --cytoDilation 3 --segmentCytoplasm segmentCytoplasm --nucleiRegion localThreshold' \
    --mcquant-opts  '--masks cellRing.ome.tif' \
    --stop-at       quantification

```

## Step: 4

Create a `markers.csv` file in the following format and save a copy within each `image folder`

```
cycle,marker_name
1,DNA_1
1,secondary_488
1,secondary_555
1,secondary_647
2,DNA_2
2,CCR4/7
2,background_555
2,background_647
3,DNA_3
3,CD163
3,CD19
3,CD3d
```

## Step: 5

Submit the job !!!

Type the following from the directory where you had saved the `submission.sh` script.

```
bash submission.sh -p '/n/scratch3/users/a/ajn16/abval' -t '/n/scratch3/users/a/ajn16/test/batch.sh'
```

where,  
`p` is path to the main/dataset folder.  
`t` is path to the template file that you just created.


##### type `sacct` into the terminal to see the jobs submitted.